In [ ]:

import os
import sys

"""
このnotebookがbaseフォルダを読めるようになるための処理
"""

root_rel = '../../'

# 相対パスを絶対パスに変換してsys.pathに追加
root_abs = os.path.abspath(root_rel)
if root_abs not in sys.path:
    sys.path.append(root_abs)
print(root_abs)


In [ ]:

# ライブラリの取り込み
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from tqdm import tqdm

import shutil
from scipy.ndimage import rotate


# Speクラスの読み込み

In [ ]:

from model.Temperature.Spe import SpeFile


In [ ]:
# spe_movieの作成メソッド
# NOTE ffmpegが必要です(多分) それ関連のエラーが出たら誰かに聞いてください
def make_spe_movie(spe: SpeFile, save_folder: str):
    save_mp4_path = os.path.join(save_folder, f"img_{spe.basename}{spe.filenum}.mp4")
    time_arr = np.arange(spe.num_frames) / spe.framerate
    time_arr = np.round(time_arr, 2) # 小数点2桁まで拾う
    
    if not os.path.exists(save_mp4_path):
        os.makedirs(os.path.dirname(save_mp4_path), exist_ok=True) # 自動的にフォルダを作るので注意
        fig, ax = plt.subplots(dpi=300)
        ax.set_xlabel('Pixel (correspond to Wavelength)')
        ax.set_ylabel('Pixel (correspond to Position)')
        vmax = spe.img.max() # 全体強度の最大値を保存しておく
        
        ims = []
        for i in tqdm(range(spe.num_frames)):
            data = pd.DataFrame(spe.img[i])
            title = ax.text(x=0.5, y=1.02,
                            s=f"{spe.basename}{spe.filenum} / {spe.OD}\nTime = {time_arr[i]:6.2f} s / Frame = {i: >4}",
                            ha='center',
                            va='bottom',
                            transform=ax.transAxes,
                            fontsize='large')
            img_plot = ax.imshow(data, animated=True, vmin=0, vmax=vmax, cmap='jet')
            if i == 0:
                fig.colorbar(img_plot, ax=ax, label='Intensity (a.u.)')
            ims.append([img_plot, title])
        print("for finish")
        ani = animation.ArtistAnimation(fig, ims, interval=100, blit=True)
        print("ani finish")
        writer = animation.FFMpegWriter(fps=10)
        ani.save(save_mp4_path, writer=writer)
        del ani
        print("save finish")
        print(f"保存先: {save_mp4_path}")
    else:
        print(f"すでに\n\t{save_mp4_path}\nに作成済み")


# ファイルpathの設定

## フォルダに入れたものを一括で回転させる時

In [ ]:
"""
複数のファイルを同じ角度で一括で回転させる時
"""
# オリジナル
before_folder = "/Volumes/T7 Touch/data/rotated_spe/before/"

# 回転したあとを保存する
after_folder = "/Volumes/T7 Touch/data/rotated_spe/after/"

# spe_movieの保存先
movie_folder = "/Volumes/T7 Touch/data/rotated_spe/spe_movie/"

In [ ]:
print("Before folder: ", before_folder)
before_files = os.listdir(before_folder)

for i, file in enumerate(before_files):
    print(f"{i+1}: {file}")

In [ ]:
# ファイルへのアクセス方法
os.path.join(before_folder, before_files[0])

# ファイルをコピーする

In [ ]:
before_spe_list = []

for files in before_files:
    spe_path = os.path.join(before_folder, files)
    before_spe_list.append(SpeFile(filepath=spe_path))

In [ ]:
before_spe_list

In [ ]:
# 保存先path
spe = before_spe_list[0]
os.path.join(after_folder, spe.OD, f"rotated_{spe.basename + str(spe.filenum)}.spe")

In [ ]:
copied_spe_list = []

# WARNING このセルを実行すると、オリジナルで再度初期化されます。回転したあとは注意!

for i, spe in enumerate(before_spe_list):
    # speはオリジナルから作成されたSpeFileオブジェクト
    print(f"{i+1}: {spe}")
    after_path = os.path.join(after_folder, spe.OD, f"rotated_{spe.basename + str(spe.filenum)}.spe")
    os.makedirs(os.path.dirname(after_path), exist_ok=True) # 自動的にフォルダを作るので注意

    shutil.copyfile(src=spe.filename, dst=after_path)
    copied_spe_list.append(SpeFile(filepath=after_path)) # SpeFileオブジェクトにしておく

In [ ]:
# コピーされた後のファイル名を見て確認しておく


In [ ]:
# オリジナルでspe movieを作っておく
# ffmpeg関連でエラー出たらコメントアウトしてください。回転後でもmovie作るセルがあるのでそちらもコメントアウトしてください。
# 参考: 石澤PCで11個movieを作ろうとしたら 25min 45sec かかった
# for i, spe in enumerate(before_spe_list):
#     print(f"\n{i+1}: {spe.filename}")
#     make_spe_movie(spe,
#                    save_folder = os.path.join(movie_folder, 'before'))

# 回転角度を設定する

In [ ]:
rotate_deg = 0.5

In [ ]:
# 関数定義
def rotate_spe(before_spe: SpeFile,
               copied_spe: SpeFile,
               rotate_deg: float):
    spe_file_id = open(copied_spe.filename, "r+b")
    # FIXME これは岡崎がSpeFileに実装してくれたのであとで消す。
    data_type_dict = {
        0: np.float32,
        1: np.int32,
        2: np.int16,
        3: np.uint16,
        8: np.uint32
    }
    img_type = data_type_dict[before_spe._data_type]
    
    xdim = np.int64(_read_at(spe_file_id, pos=42, size=1, ntype=np.int16))[0]
    ydim = np.int64(_read_at(spe_file_id, pos=656, size=1, ntype=np.int16))[0]
    img_size = xdim * ydim
    num_frames = _read_at(spe_file_id, pos=1446, size=1, ntype=np.int32)[0]
    
    # saturation_flag = False # これも保存した方が良いんだろうなと思うが、重要性は低い
    
    # 書き換える位置にシークし、imgデータを書き込む
    pos = 4100 # 1個目の位置
    for i in tqdm(range(num_frames)):
        spe_file_id.seek(pos)
        if before_spe.img[i].max() < 65_000: # サチってなければ回転。閾値が65535でないのはそういう時があるから。この数字はなんとなく
            new_img = rotate(before_spe.img[i], rotate_deg, reshape=False) # 回転
            new_img = new_img.reshape(img_size, 1) # 2次元データを1次元に
            new_img = new_img.astype(dtype=img_type)
        else: # サチってれば0にする
            # saturation_flag = True # これも保存した方が良いんだろうなと思うが、重要性は低い
            new_img = np.zeros(before_spe.img[i].shape, dtype=img_type)
        spe_file_id.write(new_img.tobytes()) # バイナリ書き込み
        pos = spe_file_id.tell() # 書き込み終了したところのposに更新する
    
    spe_file_id.close()

def _read_at(file_id, pos, size, ntype):
    pos = int(pos)
    size = int(size)
    file_id.seek(pos)
    return np.fromfile(file_id, ntype, size)


In [ ]:
# 確認
# ここで確認してることはコピーした時に確認するべきことではある気がする

def get_file_name(spe: SpeFile): # これSpeFileクラスにないっけ？ありそう
    return spe.basename + str(spe.filenum)

# 数が等しいことを確認
assert len(before_spe_list) == len(copied_spe_list)

# それぞれのファイル名が等しいことを確認
for i in range(len(before_spe_list)):
    assert get_file_name(before_spe_list[i]) == get_file_name(copied_spe_list[i])

print("OK")

In [ ]:
# 回転させる
for i in range(len(copied_spe_list)):
    rotate_spe(before_spe_list[i], copied_spe_list[i], rotate_deg)

In [ ]:
# ここでコピーしたspeファイルをもう一度インスタンス化して確認する
after_spe_list = []

for files in copied_spe_list: # コピーした時に作ったインスタンスを参照
    after_spe_list.append(SpeFile(filepath=files.filename))

In [ ]:
# imshowして確認
"""
・適切に回転 -> ロンギヌスの槍みたいになる
・サチ      -> 片方サチって回転後は0埋めしているので確認できない。frame変える
・0        -> 温度が低すぎて測温できてない。frame変える
"""
frame = -50 # 最後から50 frame

for i in range(len(before_spe_list)):
    fig  = plt.figure()
    plt.imshow(before_spe_list[i].img[frame] - after_spe_list[i].img[frame])
    plt.colorbar()
    plt.suptitle(get_file_name(before_spe_list[i]))

In [ ]:
# 回転後でspe movieを作っておく
# 参考: 石澤PCで11個movieを作ろうとしたら 25min 45sec かかった
# for i, spe in enumerate(after_spe_list):
#     print(f"\n{i+1}: {spe.filename}")
#     make_spe_movie(spe,
#                    save_folder = os.path.join(movie_folder, 'before'))
